In [ ]:
import torch.nn as nn # 파이토치의 신경망 모듈(nn)을 임포트
import torch # torch, torch.optim, torch.nn.functional 모듈을 임포트하는데 PyTorch에서 모델을 구성하고 최적화하는 데 필요한 함수와 클래스를 제공
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [4]:
# FashionMNIST을 로드하고 설정
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # 이미지를 정규화함
])

In [5]:
# FashionMNIST 훈련 및 테스트 데이터셋 다운
train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [6]:
class pyCNN(nn.Module):
    def __init__(self):
        super(pyCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)  # 과제 조건 중 첫 번째 컨볼루션 레이어
        self.pool = nn.MaxPool2d(2, 2)  # 과제 조건 중 첫번째 풀링레이어
        self.conv2 = nn.Conv2d(16, 16, 3, padding=1)  # 과제 조건 중 두 번째 컨볼루션 레이어
        self.pool2 = nn.MaxPool2d(2, 2)  # 과제 조건 중 두번째 풀링레이어
        self.fc1 = nn.Linear(16 * 7 * 7, 120)  # 과제 조건 중 첫 번째 완전 연결 레이어
        self.fc2 = nn.Linear(120, 10)  # 과제 조건 중 두 번째 완전 연결 레이어
        
def forward(self, x): # 모델의 순전파를 정의하는 메서드로 입력 텐서 x가 모델을 통과하는 과정을 설명
        x = self.pool(F.relu(self.conv1(x)))  # 첫 번째 컨볼루션 레이어와 풀링 레이어를 통과
        x = self.pool2(F.relu(self.conv2(x)))  # 두 번째 컨볼루션 레이어와 풀링 레이어를 통과
        x = x.view(-1, 16 * 7 * 7)  # 평탄화
        x = F.relu(self.fc1(x))  # 첫 번째 완전 연결 레이어를 통과
        x = self.fc2(x)  # 두 번째 완전 연결 레이어를 통과
        return x
    

In [7]:
# 초기화 진행
model = pyCNN()


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) 

In [1]:
# PyTorch를 사용하여 신경망 모델을 학습하고 검증하는 과정을 구현
def train_model(model, train_loader, test_loader, criterion, optimizer, epochs=5): # 함수를 정의하여 모델, 학습 데이터 로더, 테스트 데이터 로더, 그리고 에폭 수를 매개변수로 받음
    history = {'train_loss': [], 'train_acc': [], 'test_loss': [], 'test_acc': []} 

    # 에폭 수만큼 반복하면서 모델의 학습 및 평가를 수행하는데 학습 시에는 model.train()을 호출하여 모델을 학습 모드로 설정하고, 평가 시에는 model.eval()을 호출하여 평가 모드로 설정
    for epoch in range(epochs):
        # 모델의 학습
        model.train()
        train_loss = 0
        train_correct = 0
        total_train = 0
        
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            train_correct += (predicted == labels).sum().item()
            total_train += labels.size(0)
        
        # 모델의 평가
        model.eval()
        test_loss = 0
        test_correct = 0
        total_test = 0
        with torch.no_grad():
            for images, labels in test_loader:
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                test_loss += loss.item() * images.size(0) # 정확도와 손실을 기록, 진행 상황 모니터링
                _, predicted = torch.max(outputs.data, 1)
                test_correct += (predicted == labels).sum().item()
                total_test += labels.size(0)
        
        # 평균 손실과 정확도
        epoch_train_loss = train_loss / total_train
        epoch_train_acc = 100 * train_correct / total_train
        epoch_test_loss = test_loss / total_test
        epoch_test_acc = 100 * test_correct / total_test
        
        # history에 저장
        history['train_loss'].append(epoch_train_loss)
        history['train_acc'].append(epoch_train_acc)
        history['test_loss'].append(epoch_test_loss)
        history['test_acc'].append(epoch_test_acc)
        
        print(f'Epoch {epoch + 1}/{epochs}: Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.2f}%, Test Loss: {epoch_test_loss:.4f}, Test Acc: {epoch_test_acc:.2f}%')
    
    return history # 학습과 테스트 과정에서의 손실과 정확도를 기록한 history 딕셔너리를 반환

In [10]:
# 신경망 모델을 학습시키고 그 결과를 처리하는 과정을 보여줌
def plot_metrics(history):
    plt.figure(figsize=(10, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['test_loss'], label='Validation Loss')
    plt.title('Loss Over Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'], label='Train Accuracy')
    plt.plot(history['test_acc'], label='Validation Accuracy')
    plt.title('Accuracy Over Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# 함수를 호출하여 신경망 모델을 학습시키는데 총 5 에폭(epoch) 동안 학습을 수행하고 학습 과정에서 계산된 손실과 정확도 같은 통계 정보를 history 변수에 저장
history = train_model(model, train_loader, test_loader, criterion, optimizer, epochs=5)
plot_metrics(history) # plot_metrics 함수를 호출하여 history에 저장된 학습 결과 시각화

: 

In [51]:
num_epochs = 5  # 예시로 5로 설정
total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # 순전파
        outputs = model.forward(images)  # 모델 호출 방식 수정
        probabilities = F.softmax(outputs, dim=1)  # 소프트맥스 함수 적용
        
        # 손실 계산
        loss = criterion(outputs, labels)
        
        # 역전파 및 최적화
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # 훈련 과정에서의 손실 및 정확도 모니터링
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_steps}], Loss: {loss.item():.4f}')


NotImplementedError: Module [pyCNN] is missing the required "forward" function